In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import shutil

In [4]:
print("version of tensorflow {}".format(tf.__version__))

version of tensorflow 1.15.2


In [0]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:]
LABEL = CSV_COLUMNS[0]

In [0]:
df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

In [0]:
def train_input_fn(df,num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
      x=df,
      y=df[LABEL],
      num_epochs=num_epochs,
      shuffle=True,
      queue_capacity= 1000
  )

In [0]:
def valid_input_fn(df,num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
      x=df,
      y=df[LABEL],
      num_epochs=num_epochs,
      shuffle=False,
      queue_capacity= 1000
  )


In [0]:
def Test_input_fn(df,num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

In [0]:
#feature column
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)

OUTPUT_DIR="taxi_trained"
shutil.rmtree(OUTPUT_DIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(feature_columns=make_feature_cols(),model_dir=OUTPUT_DIR)

In [12]:
model.train(input_fn=train_input_fn(df_train,10))

In [0]:
def print_rmse(model, df):
  metrics = model.evaluate(valid_input_fn(df,1))
  print(metrics)
  print("RMSE {}".format(np.sqrt(metrics["average_loss"])))

In [14]:
print_rmse(model,df_valid)

{'average_loss': 109.51098, 'label/mean': 11.666427, 'loss': 13023.984, 'prediction/mean': 10.841303, 'global_step': 608}
RMSE 10.464749336242676


In [15]:
predictions = model.predict(input_fn=Test_input_fn(df_test,1))

for items in predictions:
  print(items)

{'predictions': array([10.797019], dtype=float32)}
{'predictions': array([10.794561], dtype=float32)}
{'predictions': array([10.795457], dtype=float32)}
{'predictions': array([10.793044], dtype=float32)}
{'predictions': array([10.796909], dtype=float32)}
{'predictions': array([10.796714], dtype=float32)}
{'predictions': array([10.795357], dtype=float32)}
{'predictions': array([10.795429], dtype=float32)}
{'predictions': array([10.7972145], dtype=float32)}
{'predictions': array([10.795066], dtype=float32)}
{'predictions': array([10.797355], dtype=float32)}
{'predictions': array([10.797557], dtype=float32)}
{'predictions': array([10.79098], dtype=float32)}
{'predictions': array([10.79486], dtype=float32)}
{'predictions': array([10.821589], dtype=float32)}
{'predictions': array([10.795723], dtype=float32)}
{'predictions': array([10.796616], dtype=float32)}
{'predictions': array([10.822747], dtype=float32)}
{'predictions': array([10.797019], dtype=float32)}
{'predictions': array([10.794258

In [16]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTPUT_DIR,ignore_errors=True)
model = tf.estimator.DNNRegressor(hidden_units=[32 ,8 ,2],
                                  feature_columns=make_feature_cols(),model_dir=OUTPUT_DIR)
model.train(input_fn=train_input_fn(df_train,100))
print_rmse(model,df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fda0199b7b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:

In [23]:
from google.cloud import bigquery
import numpy as np
import pandas as pd


ImportError: ignored

In [0]:
def create_query(phase,N):
  
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if N==None:
    if phase<2:
      #Training
      query = "{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime as string)), 4))< 2".format(base_query)

    else:
      #validation
      query ="{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 4)) = {1}".format(base_query, phase)
  else:
    query = "{0} AND ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), {1})) = {2}".format(base_query, N, phase)
    
  return query

In [0]:
query = create_query(2, 100000)

In [21]:
df = bigquery.Client().query(query).to_dataframe()

NameError: ignored

In [22]:
!pip install --upgrade googleapis-common-protos

Requirement already up-to-date: googleapis-common-protos in /usr/local/lib/python3.6/dist-packages (1.51.0)


In [28]:
!pip install --upgrade pydata-google-auth google-cloud-bigquery[pandas]

Requirement already up-to-date: pydata-google-auth in /usr/local/lib/python3.6/dist-packages (0.3.0)
     |████████████████████████████████| 174kB 3.4MB/s 
ERROR: tensorflow 1.15.2 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.14.0 which is incompatible.
ERROR: google-cloud-storage 1.18.1 has requirement google-resumable-media<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.0 which is incompatible.
ERROR: google-cloud-bigquery 1.24.0 has requirement google-auth<2.0dev,>=1.9.0, but you'll have google-auth 1.7.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12

In [1]:
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=YZ5yhgsGjOQI6dQC94UUbO4e02YoQX&prompt=consent&access_type=offline
Enter the authorization code: 4/yAGW43_6MO55Mur_ft4DB9HjGt4zdjWcow3pYZcHa9lh265B8s9Jnjg


In [24]:
print_rmse(model, df)

NameError: ignored